# 암호화 대비 SQL 데이터 출력 클래스 제작

## 구현 방식 1 : 기존 클래스 상속
기존 SQL 접속용 클래스 PreprocessQuery를 상속한 SelectData 클래스 구현

In [1]:
import pymysql
from sqlalchemy import create_engine
from abc import ABC, abstractmethod
from typing import *
import re

from prettytable import PrettyTable
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

### 기존 PreprocessQuery 클래스

In [2]:
class PreprocessQuery:
    """데이터 전처리 목적의 SQL 쿼리 클래스"""
    def __init__(self, pw):
        self._pw = pw
        self.connection = None
        self.Cursor = None
        self.SQL = None
    
    def connectMySQL(self, 
                     serverIP: str, port_num: int, user_name: str, database_name: str, kr_encoder: str):
        """MySQL DBMS 데이터베이스에 접속: 서버IP주소, 사용자명, 계정 암호, 데이터베이스명, 한글 인코딩 방식"""
        try:
            self.connection = pymysql.connect(
                host = serverIP, port = port_num,
                user = user_name, password = self._pw,
                db = database_name, charset = kr_encoder
            )
            self.Cursor = self.makeCursor(self.connection)
        except pymysql.Error as e:
            print(f"Error Connecting to MySQL from Python: {e}")
    
    def makeCursor(self, connect):
        """커서 생성"""
        return connect.cursor()
    
    def dataQueryLanguage(self, sql):
        """SQL 쿼리문 작성"""
        self.SQL = f"{sql}"
    
    def queryExecute(self):
        """SQL 쿼리문 실행 및 예외처리"""
        try:
            self.Cursor.execute(self.SQL)
            actionOutput = self.Cursor.fetchall()
            return actionOutput
        except pymysql.Error as e:
            print(f"Error Executing Query: {e}")
    
    def queryCommit(self):
        """실행 결과 확정"""
        self.Cursor.execute(self.SQL)
        self.Cursor.commit()
    
    def closeConnection(self):
        """연결 및 커서 닫기"""
        if self.Cursor:
            self.Cursor.close()
        if self.connection:
            self.connection.close()

### SelectData 클래스 : 테이블 및 컬럼 단위로 데이터 묶기

In [3]:
class SelectData(PreprocessQuery):
    """SQL로 데이터 출력 이후, 출력된 데이터를 테이블 단위로 조인하거나, 컬럼별로 묶는 클래스"""
    def __init__(self, data):
        self._data = data
        self.connection = None
        self.Cursor = None
        self._pw = "1234"
        
    def printTable(self, data):
        """테이블 출력 결과에 PrettyTable 적용하는 함수"""
        if data:
            columns = [ [desc[0] for desc in self.Cursor.description] ]
            table = PrettyTable(*columns)
    
            for row in data:
                row_list = list(row)
                table.add_row(row_list)
            return table
        return False
        
    def showData(self, table):
        """테이블 50줄을 select 하여 데이터 형태를 파악할 수 있도록 하는 함수"""
        self.connectMySQL(serverIP = "localhost", port_num = 3306, user_name = "root", database_name = self._data, kr_encoder = "utf8")
        self.dataQueryLanguage(f"SELECT * FROM {table} LIMIT 0, 50")
        results = self.queryExecute()
        table = self.printTable(results)
        return table
        
    def joinColunmns(self, tables: list, columns: list):
        """원하는 컬럼들로 데이터 조인해주는 함수.
            구현할 조인의 형태를 고민해야 함."""
        pass
        
    def concatColumns(self, columns: list, table: str):
        """원하는 컬럼들을 전부 합치는 함수 (예: 김민주 / 2022-10-23 -> 김민주20221023)
           CONCAT 구현 정교화 필요."""    
        self.connectMySQL(serverIP = "localhost", port_num = 3306, user_name = "root", database_name = self._data, kr_encoder = "utf8")
        self.dataQueryLanguage(f"SELECT CONCAT ({columns}) AS result FROM {table}")
        results = self.queryExecute()
        table = self.printTable(results)
        return table
        
    
d = SelectData(data = "FINANCIALCONSUMER")

In [4]:
d.showData(table = "DATA_JOIN_ACCOMODATIONAPP")

NUM_SERIAL,NAME,BIRTH_DATE,GENDER,EMAIL,PHONE_NUMBER,TF_BUSINESS_MEMBER,USE_SERVICE_AREA,USE_SERVICE_DATE
1,고대옥,1966-04-01,female,고대옥@gmail.com,010-5978-9544,N,전라북도 익산시 서동로 464(용제동),2021-07-20
2,사미소,1991-01-12,female,사미소@outlook.com,010-3920-5092,N,제주특별자치도 제주시 조천읍 신촌남8길 87,2020-06-06
3,서회걸,1950-09-20,male,서회걸@naver.com,010-7209-1680,Y,경상북도 영천시 신녕면 찰방길 28,2022-02-17
4,장예홍,2002-11-22,male,장예홍@naver.com,010-3093-9161,Y,경기도 성남시 수정구 시민로 172(신흥동),2021-11-04
5,표재창,1983-08-21,male,표재창@nate.com,010-2443-6530,Y,경상남도 의령군 지정면 기강로2길 5,2020-08-20
6,주동지,1981-09-11,female,주동지@naver.com,010-4437-3442,Y,경기도 이천시 장호원읍 서동대로8369번길 166-13,2020-11-13
7,방선두,1954-03-22,male,방선두@naver.com,010-8876-2788,Y,강원특별자치도 태백시 피내골길 11(철암동),2022-03-20
8,송민석,1956-02-23,male,송민석@daum.net,010-6455-7721,Y,대구광역시 동구 동북로 415-3(신암동),2020-06-25
9,우기모,1997-12-30,female,우기모@nate.com,010-2763-0443,Y,충청북도 옥천군 안내면 용촌차정로 174-10,2020-12-30
10,황보년,1958-10-19,male,황보년@daum.net,010-7686-0661,N,경상북도 김천시 충효8길 2(성내동),2021-06-09


In [5]:
columns = ['NUM_SERIAL', 'NAME', 'BIRTH_DATE']

In [6]:
d.concatColumns(columns, 'DATA_JOIN_ACCOMODATIONAPP')

Error Executing Query: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '['NUM_SERIAL', 'NAME', 'BIRTH_DATE']) AS result FROM DATA_JOIN_ACCOMODATIONAPP' at line 1")


False

## 구현 방식 2 : SQL 데이터 전처리 통합 클래스 작성
기존 PreprocessQuery와 SelectData를 합쳐서, SQL 접속부터 조인 쿼리까지 통합한 클래스 작성

# 암호화 클래스 제작

In [7]:
import os
from hashlib import blake2b
from hashlib import sha256

In [8]:
class EncryptData:
    def __init__(self, data):
        self._data = data
        
    def addSalt(self, data):
        """BLAKE2B 알고리즘으로 데이터에 Salt값을 붙이는 함수"""
        results = []
        
        for i in range(len(data)):
            d = data[i].encode('utf-8')
            salt = os.urandom(blake2b.SALT_SIZE)
            h = blake2b(salt = salt)
            updateData = h.update(d)
            results.append(updateData)
            
        return results
        
    def makeHash(self, data):
        """데이터 합치기, Salt값 더하기 작업 완료된 데이터를 SHA256 알고리즘으로 해싱하는 함수"""
        results = []
        
        for i in range(len(data)):
            d = data[i].encode('utf-8')
            result = sha256(d)
            results.append(result)
            
        return results
    
    def printResult(self, data):
        """해싱 완료된 민감정보 데이터를 같은 인덱스의 다른 데이터와 합쳐서 프린트하는 함수"""
        pass

In [9]:
d2 = "1, '고대옥', datetime.date(1966, 4, 1), 'female', '고대옥@gmail.com', '010-5978-9544', 'N', '전라북도 익산시 서동로 464(용제동)', datetime.date(2021, 7, 20)"

In [13]:
d3 = EncryptData(d2)